<div dir='rtl'>
    
# الوصف

</div>
    
<div dir='rtl'>
يُستخدم هذا الدفتر لطلب قائمة بالخرائط النقطية المتاحة (مع معرف البيانات النقطية) لطبقة WaPOR محددة نظرًا لفترة استخدام WaPOR API
</div>    
    

In [1]:
import requests
import pandas as pd

path_query=r'https://io.apps.fao.org/gismgr/api/v1/query/'

<div dir='rtl'>

# الخطوة 1: اكتب حمولة الاستعلام    
    
</div>



In [2]:
workspace='WAPOR_2'
cube_code='L1_T_A'
time_range="[2011-01-01,2014-01-01]"

In [3]:
# الحصول على قياس مكعب البيانات
cube_url=f'https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/{workspace}/cubes/{cube_code}/measures'
resp=requests.get(cube_url).json()
measure=resp['response']['items'][0]['code']
measure

'WATER_MM'

In [4]:
# احصل على البعد الزمني لمكعب البيانات
cube_url=f'https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/{workspace}/cubes/{cube_code}/dimensions'
resp=requests.get(cube_url).json()
items=pd.DataFrame.from_dict(resp['response']['items'])
dimension=items[items.type=='TIME']['code'].values[0]
dimension

'YEAR'

In [5]:
query={
  "type": "MDAQuery_Table",
  "params": {
    "properties": {
      "metadata": True,
      "paged": False
    },
    "cube": {
      "code": cube_code,
      "workspaceCode": workspace,
      "language": "en"
    },
    "dimensions": [
      {
        "code": dimension,
        "range": time_range        
      }
    ],
    "measures": [
      measure
    ],
    "projection": {
      "columns": [
        "MEASURES"
      ],
      "rows": [
        dimension
      ]
    }
  }
}

<div dir='rtl'>

# الخطوة 2: POST طلب      
    
</div>



In [6]:
resp=requests.post(path_query,json=query)
resp_dict = resp.json() #read response JSON string
resp_dict

{'requestId': '1bdde3b3-db1e-43a5-b779-00ee3cec14c1',
 'timestamp': 1594808160026,
 'runtime': 10,
 'status': 200,
 'message': 'OK',
 'response': {'properties': {'flat': True,
   'metadata': True,
   'showEmptyRows': False,
   'paged': False,
   'pageSize': 20,
   'page': 0},
  'header': {'height': 1,
   'rows': [[{'value': 'Year',
      'type': 'ROW_COLUMN_HEADER',
      'width': 1,
      'height': 1},
     {'value': 'Amount of Water',
      'type': 'COLUMN_HEADER',
      'width': 2,
      'height': 1}]]},
  'items': [[{'value': '2011', 'type': 'ROW_HEADER', 'width': 1, 'height': 1},
    {'type': 'DATA_CELL',
     'value': 1,
     'metadata': {'raster': {'id': 'L1_T_11',
       'bbox': [{'srid': 'EPSG:4326',
         'value': [-30.0044643, -40.0044644, 65.0044644, 40.0044643]}]}}},
    {'type': 'DATA_CELL',
     'value': 1,
     'metadata': {'raster': {'id': 'L1_T_11',
       'bbox': [{'srid': 'EPSG:4326',
         'value': [-30.0044643, -40.0044644, 65.0044644, 40.0044643]}]}}}],
   

<div dir='rtl'>
    
# الخطوة 3: تحويل نص الاستجابة إلى DataFrame
    
</div>

In [7]:
items=resp_dict['response']['items']
items[0] # الحصول على العنصر الأول

[{'value': '2011', 'type': 'ROW_HEADER', 'width': 1, 'height': 1},
 {'type': 'DATA_CELL',
  'value': 1,
  'metadata': {'raster': {'id': 'L1_T_11',
    'bbox': [{'srid': 'EPSG:4326',
      'value': [-30.0044643, -40.0044644, 65.0044644, 40.0044643]}]}}},
 {'type': 'DATA_CELL',
  'value': 1,
  'metadata': {'raster': {'id': 'L1_T_11',
    'bbox': [{'srid': 'EPSG:4326',
      'value': [-30.0044643, -40.0044644, 65.0044644, 40.0044643]}]}}}]

In [8]:
values=[item[0]['value'] for item in items]
rasterId=[item[1]['metadata']['raster']['id'] for item in items]
bbox=[item[1]['metadata']['raster']['bbox'] for item in items]

df=pd.DataFrame({dimension:values,
                'rasterId':rasterId,
                'bbox':bbox})
df

,YEAR,rasterId,bbox
0,2011,L1_T_11,"[{'srid': 'EPSG:4326', 'value': [-30.0044643, ..."
1,2012,L1_T_12,"[{'srid': 'EPSG:4326', 'value': [-30.0044643, ..."
2,2013,L1_T_13,"[{'srid': 'EPSG:4326', 'value': [-30.0044643, ..."
3,2014,L1_T_14,"[{'srid': 'EPSG:4326', 'value': [-30.0044643, ..."
